In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
def direction(val):
    val1 = Conversion().convert_int256_bytes_to_int(val, signed=False) 
    val2 = Conversion().convert_int256_bytes_to_int(val, signed=True)
    return val1 != val2

In [3]:
chain = Net.POLYGON
platform = Platform.UNIV3
contract = JSONContract.UniswapV3Pool

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Burn])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 50

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    event_signature, owner, tick_lower, tick_upper = evt["topics"]
    args = Conversion().decode_data(evt["data"])
    
    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['owner'] = Conversion().convert_uint256_hex_string_to_address(owner)
    event['details']['tick_lower'] = Conversion().convert_uint256_string_to_int(tick_lower, signed=True)
    event['details']['tick_upper'] = Conversion().convert_uint256_string_to_int(tick_upper, signed=True)    
    if(len(args) == 4):
        event['details']['liquidity_amount'] = Conversion().convert_int256_bytes_to_int(args[1])
        event['details']['amount0'] = Conversion().convert_int256_bytes_to_int(args[2])
        event['details']['amount1'] = Conversion().convert_int256_bytes_to_int(args[3])
    events[k] = event
    
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")

Swap at block:61,383,551 tx:0x4bd41bebcc454a56c651ffb13387f842514815ce07f4fcbe0ccf3cc214822bd4
Swap at block:61,383,553 tx:0x0ae3e3b88c369f4fb88517021a5bcce53deb88d65442d2057f7cfe75dc2e342a
Swap at block:61,383,558 tx:0xbd13fc88688732220e99222da7164358a4600c8917ad6a22c093ce207f50471c
Swap at block:61,383,560 tx:0x630d017e9168abc143d0650d22f2d2b0d6c47c176066156d8222c30fee03d30c
Swap at block:61,383,563 tx:0x8039cb346b49d875fb0f35202e084e6a52b537e202d7519f3243fea0e5fbdbd7
Swap at block:61,383,574 tx:0xd33fb87769af8724483beb0281c3065944b46d55c3265748cb565194477220d7
Swap at block:61,383,574 tx:0xa8dbb93013ae14f74a1d83e1b49314ab2a87fa7b66e6824dcda2a4aec780e2af
Swap at block:61,383,590 tx:0xa1c2d3e053b3c468daaa1a54e6a3e5cb22444cafb309ed2145b2b36044473152
Swap at block:61,383,590 tx:0xa1c2d3e053b3c468daaa1a54e6a3e5cb22444cafb309ed2145b2b36044473152
Swap at block:61,383,590 tx:0xa1c2d3e053b3c468daaa1a54e6a3e5cb22444cafb309ed2145b2b36044473152
Swap at block:61,383,590 tx:0xa1c2d3e053b3c468daaa

In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv3pool,burn,uniswap_v3,0x167384319b41f7094e62f7506409eb38079abff8,0x4bd41bebcc454a56c651ffb13387f842514815ce07f4...,61383551,1725384541,{'web3_type': <class 'web3._utils.datatypes.Bu...
1,polygon,uniswapv3pool,burn,uniswap_v3,0x00a59c2d0f0f4837028d47a391decbffc1e10608,0x0ae3e3b88c369f4fb88517021a5bcce53deb88d65442...,61383553,1725384547,{'web3_type': <class 'web3._utils.datatypes.Bu...
2,polygon,uniswapv3pool,burn,uniswap_v3,0x13840f430765ee56a1afc5b13c127c96708f060e,0xbd13fc88688732220e99222da7164358a4600c8917ad...,61383558,1725384557,{'web3_type': <class 'web3._utils.datatypes.Bu...
3,polygon,uniswapv3pool,burn,uniswap_v3,0x13840f430765ee56a1afc5b13c127c96708f060e,0x630d017e9168abc143d0650d22f2d2b0d6c47c176066...,61383560,1725384561,{'web3_type': <class 'web3._utils.datatypes.Bu...
4,polygon,uniswapv3pool,burn,uniswap_v3,0xa4d8c89f0c20efbe54cba9e7e7a7e509056228d9,0x8039cb346b49d875fb0f35202e084e6a52b537e202d7...,61383563,1725384567,{'web3_type': <class 'web3._utils.datatypes.Bu...
5,polygon,uniswapv3pool,burn,uniswap_v3,0xa9077cdb3d13f45b8b9d87c43e11bce0e73d8631,0xd33fb87769af8724483beb0281c3065944b46d55c326...,61383574,1725384591,{'web3_type': <class 'web3._utils.datatypes.Bu...
6,polygon,uniswapv3pool,burn,uniswap_v3,0xbb98b3d2b18aef63a3178023a920971cf5f29be4,0xa8dbb93013ae14f74a1d83e1b49314ab2a87fa7b66e6...,61383574,1725384591,{'web3_type': <class 'web3._utils.datatypes.Bu...
7,polygon,uniswapv3pool,burn,uniswap_v3,0x9ceff2f5138fc59eb925d270b8a7a9c02a1810f2,0xa1c2d3e053b3c468daaa1a54e6a3e5cb22444cafb309...,61383590,1725384625,{'web3_type': <class 'web3._utils.datatypes.Bu...
8,polygon,uniswapv3pool,burn,uniswap_v3,0x9ceff2f5138fc59eb925d270b8a7a9c02a1810f2,0xa1c2d3e053b3c468daaa1a54e6a3e5cb22444cafb309...,61383590,1725384625,{'web3_type': <class 'web3._utils.datatypes.Bu...
9,polygon,uniswapv3pool,burn,uniswap_v3,0x9ceff2f5138fc59eb925d270b8a7a9c02a1810f2,0xa1c2d3e053b3c468daaa1a54e6a3e5cb22444cafb309...,61383590,1725384625,{'web3_type': <class 'web3._utils.datatypes.Bu...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'burn',
  'platform': 'uniswap_v3',
  'address': '0x167384319b41f7094e62f7506409eb38079abff8',
  'tx_hash': '0x4bd41bebcc454a56c651ffb13387f842514815ce07f4fcbe0ccf3cc214822bd4',
  'blk_num': 61383551,
  'timestamp': 1725384541,
  'details': {'web3_type': web3._utils.datatypes.Burn,
   'owner': '0xC36442b4a4522E871399CD717aBDD847Ab11FE88',
   'tick_lower': -87720,
   'tick_upper': -87120}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'burn',
  'platform': 'uniswap_v3',
  'address': '0x00a59c2d0f0f4837028d47a391decbffc1e10608',
  'tx_hash': '0x0ae3e3b88c369f4fb88517021a5bcce53deb88d65442d2057f7cfe75dc2e342a',
  'blk_num': 61383553,
  'timestamp': 1725384547,
  'details': {'web3_type': web3._utils.datatypes.Burn,
   'owner': '0xC36442b4a4522E871399CD717aBDD847Ab11FE88',
   'tick_lower': 120731,
   'tick_upper': 121012}},
 2: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'burn',
  'platf